In [1]:
# example of defining composite models for training cyclegan generators
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.initializers import RandomNormal
from keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils.vis_utils import plot_model

import os
import random
from random import randint
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
plt.style.use("grayscale")
get_ipython().run_line_magic('matplotlib', 'inline')
#%matplotlib notebook
import time

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from skimage import color

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.losses import mean_absolute_error
import keras.backend as K
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.backend import clear_session


from keras.models import Input

from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [10]:
# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_image = Input(shape=image_shape)
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	# define model
	model = Model(in_image, patch_out)
	# compile model
	model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
	return model
 
# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g
 
# define the standalone generator model
def define_generator(image_shape, n_resnet=9,size=3):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(256, g)
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(size, (7,7), padding='same', kernel_initializer=init)(g)
	out_image = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model


 

In [200]:
# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1_13, d_model, g_model_2, image_shape_c,image_shape_t,size1,size2):
	# ensure the model we're updating is trainable
	g_model_1_13.trainable = True
	# mark discriminator as not trainable
	d_model.trainable = False
	# mark other generator model as not trainable
	g_model_2.trainable = False
	# discriminator element
	input_gen = Input(shape=image_shape_t)
	gen1_out_3 = g_model_1_13(input_gen)
	output_d = d_model(gen1_out_3)
# 	# identity element
# 	input_id = Input(shape=image_shape_c)
# 	output_id = g_model_1_13(input_id)
	# forward cycle
	output_f = g_model_2(gen1_out_3)
	# backward cycle
# 	gen2_out = g_model_2(input_id)
# 	output_b = g_model_1(gen2_out)
	# define model graph
	model = Model([input_gen], [output_d, output_f])
	# define optimization algorithm configuration
	opt = Adam(lr=0.0002, beta_1=0.5)
	# compile model with weighting of least squares loss and L1 loss
	model.compile(loss=['mse', 'mae'], loss_weights=[1, 10], optimizer=opt)
	return model
 
# # input shape
# image_shape_c = (256,256,3)
# image_shape_t = (256,256,1)
# # generator: A -> B
# g_model_AtoB = define_generator(image_shape)
# # generator: B -> A
# g_model_BtoA = define_generator(image_shape)
# # discriminator: A -> [real/fake]
# d_model_A = define_discriminator(image_shape)
# # discriminator: B -> [real/fake]
# d_model_B = define_discriminator(image_shape)
# # composite: A -> B -> [real/fake, A]
# c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# # composite: B -> A -> [real/fake, B]
# c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

In [195]:
image_shape_c = (256,256,3)
image_shape_t = (256,256,1)
# A thermal
# B rgb
# generator: A -> B
g_model_thermal_to_rgb = define_generator(image_shape_t, n_resnet=9,size= 3)
# generator: B -> A
g_model_rgb_to_thermal = define_generator(image_shape_c, n_resnet=9,size= 1)
# discriminator: A -> [real/fake]
d_model_thermal = define_discriminator(image_shape_t)
# discriminator: B -> [real/fake]
d_model_rgb = define_discriminator(image_shape_c)
# composite: A -> B -> [real/fake, A]
c_model_thermal_to_rgb = define_composite_model(g_model_thermal_to_rgb, d_model_rgb, g_model_rgb_to_thermal, image_shape_c,image_shape_t,size1=1,size2 = 3)
# composite: B -> A -> [real/fake, B]
c_model_rgb_to_thermal = define_composite_model(g_model_rgb_to_thermal, d_model_thermal, g_model_thermal_to_rgb, image_shape_t,image_shape_c,size1=3,size2 = 1)

In [104]:
def generate_real_samples_t(train_crops, n_samples, patch_shape):
    # retrieve selected images
    x, y = next(train_crops)    
    if len(x.shape) != 4:
        x = np.expand_dims(x, axis=0)
    z = np.ones((n_samples, patch_shape, patch_shape, 1))
    return [x, z]

def generate_real_samples_c(train_crops, n_samples, patch_shape):
    # retrieve selected images
    x, y = next(train_crops)    
    if len(x.shape) != 4:
        y = np.expand_dims(y, axis=0)
    z = np.ones((n_samples, patch_shape, patch_shape, 1))
    return [y, z]
# select a batch of random samples, returns images and target

# def generate_real_samples(dataset, n_samples, patch_shape):
# 	# choose random instances
# 	ix = randint(0, dataset.shape[0], n_samples)
# 	# retrieve selected images
# 	X = dataset[ix]
# 	# generate 'real' class labels (1)
# 	y = ones((n_samples, patch_shape, patch_shape, 1))
# 	return X, y

In [19]:
# # generate a batch of images, returns images and targets
# def generate_fake_samples(g_model, dataset, patch_shape):
# 	# generate fake instance
# 	X = g_model.predict(dataset)
# 	# create 'fake' class labels (0)
# 	y = zeros((len(X), patch_shape, patch_shape, 1))
# 	return X, y
def generate_fake_samples(g_model, m, patch_shape):
    # generate fake instance
#     with graph.as_default():
    X = g_model.predict(m)
        # create 'fake' class labels (0)
    z = zeros((len(X), patch_shape, patch_shape, 1))
    return X, z

In [6]:
from functions import *
im_width = 640; im_height = 512
border = 5
path_train = '../../../data/FLIR/training/Calib_cut/'
# path_train = '/home/nvs/Documents/DB/FLIR/Calib/Sample2/'
#path_train = '/home/nadav.shalev/Desktop/Calib/'
X,Y = get_data(path_train, im_height, im_width)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=16/1040,random_state=2019)

/opt/anaconda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [16]:
ir_name = 'monday_schedualer_10.12'
weight_name = '0.02vgg_(1,1000)w_30e3iter_8b'
batch_size = 1
data_gen_args = dict(horizontal_flip=True)
train_crops = create_augment(X_train, 
               y_train, 
               dict_augments=data_gen_args, 
               batch_size=batch_size, 
               shuffle=True, 
               rand_crop=True, 
               seed=1234)
plot_crops = create_augment(X_test, 
               y_test, 
               dict_augments=data_gen_args, 
               batch_size=1, 
               shuffle=False, 
               rand_crop=False)



In [190]:
# train cyclegan models
# A thermal
# B rgb
def train(n_steps,d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
    #     global graph
    #     graph = tf.get_default_graph()
    #     with graph.as_default():
    K.clear_session()
    # define properties of the training run
    n_epochs, n_batch, = 1, 1
    # determine the output square shape of the discriminator
    n_patch = d_model_A.output_shape[1]
    # unpack dataset
    # 	trainA, trainB = dataset
    # prepare image pool for fakes
#     poolA, poolB = list(), list()
    # calculate the number of batches per training epoch
    # 	bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    # 	n_steps = bat_per_epo * n_epochs

    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        X_realA, y_realA = generate_real_samples_t(dataset, n_batch, n_patch)
        X_realB, y_realB = generate_real_samples_c(dataset, n_batch, n_patch)
        # generate a batch of fake samples

        X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
        a,b =              generate_fake_samples(g_model_thermal_to_rgb,X_realA,1)
        print(33)
        X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)        
        # update fakes from pool
        X_fakeA = update_image_pool(poolA, X_fakeA)
        X_fakeB = update_image_pool(poolB, X_fakeB)
        # update generator B->A via adversarial and cycle loss
        g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
        # update discriminator for A -> [real/fake]
        dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
        dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
        # update generator A->B via adversarial and cycle loss
        g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
        # update discriminator for B -> [real/fake]
        dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
        dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
        # summarize performance
        print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))

In [169]:
def generate_fake_samples(g_model, m, patch_shape):
    # generate fake instance
#     with graph.as_default():
    X = g_model.predict(m)
        # create 'fake' class labels (0)
    z = np.zeros((len(X), patch_shape, patch_shape, 1))
    return X, z

def generate_real_samples_t(train_crops, n_samples, patch_shape):
    # retrieve selected images
    x, y = next(train_crops)    
    if len(x.shape) != 4:
        x = np.expand_dims(x, axis=0)
    z = np.ones((n_samples, patch_shape, patch_shape, 1))
    return x, z

def generate_real_samples_c(train_crops, n_samples, patch_shape):
    # retrieve selected images
    x, y = next(train_crops)    
    if len(x.shape) != 4:
        y = np.expand_dims(y, axis=0)
    z = np.ones((n_samples, patch_shape, patch_shape, 1))
    return y, z

x, y = next(train_crops)
# g_model_thermal_to_rgb.predict(x)
# a = define_generator((256,256,3),size=3)
# g_model_thermal_to_rgb = define_generator(image_shape_c, n_resnet=9,size= 1)
# c = g_model_thermal_to_rgb.predict(y)
# print(c)
X_realA, y_realA = generate_real_samples_t(train_crops, 1, 1)
print(X_realA.shape)
a,b = generate_fake_samples(g_model_thermal_to_rgb,X_realA,1)

(1, 256, 256, 1)


ValueError: Tensor Tensor("activation_15/Tanh:0", shape=(?, ?, ?, 3), dtype=float32) is not an element of this graph.

In [202]:
n_patch = d_model_rgb.output_shape[1]
# manually enumerate epochs
for i in range(3):
    # select a batch of real samples
    X_realA, y_realA = generate_real_samples_t(train_crops, 1, n_patch)
    X_realB, y_realB = generate_real_samples_c(train_crops, 1, n_patch)
    X_fakeB, y_fakeB = generate_fake_samples(g_model_thermal_to_rgb, X_realA, n_patch)
    X_fakeA, y_fakeA = generate_fake_samples(g_model_rgb_to_thermal, X_realB, n_patch)   
    # update fakes from pool
#     X_fakeA = update_image_pool(poolA, X_fakeA)
#     X_fakeB = update_image_pool(poolB, X_fakeB)
    # update generator B->A via adversarial and cycle loss
    g_loss2, _, _ = c_model_rgb_to_thermal.train_on_batch([X_realB], [y_realA, X_realA])
    # update discriminator for A -> [real/fake]
    dA_loss1 = d_model_thermal.train_on_batch(X_realA, y_realA)
    dA_loss2 = d_model_thermal.train_on_batch(X_fakeA, y_fakeA)
# update generator A->B via adversarial and cycle loss
    g_loss1, _, _ = c_model_thermal_to_rgb.train_on_batch([X_realA], [y_realA, X_realA])
    # update discriminator for B -> [real/fake]
    dB_loss1 = d_model_rgb.train_on_batch(X_realB, y_realB)
    dB_loss2 = d_model_rgb.train_on_batch(X_fakeB, y_fakeB)
    # summarize performance
    print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))

ValueError: Error when checking target: expected model_19 to have shape (256, 256, 3) but got array with shape (256, 256, 1)

In [168]:
n_steps = 10
train(n_steps,d_model_thermal, d_model_rgb, g_model_thermal_to_rgb, g_model_rgb_to_thermal, c_model_thermal_to_rgb, c_model_thermal_to_rgb, train_crops)

ValueError: Tensor Tensor("activation_15/Tanh:0", shape=(?, ?, ?, 3), dtype=float32) is not an element of this graph.